In [2]:
# Python3 샘플 코드 #


import requests

url = '
params ={'serviceKey' : '', 'pageNo' : '1', 'numOfRows' : '1000', 'dataType' : 'XML', 'base_date' : '20210628', 'base_time' : '0600', 'nx' : '55', 'ny' : '127' }

response = requests.get(url, params=params)
print(response.content)

b'<?xml version="1.0" encoding="UTF-8"?>\r\n<response><header><resultCode>10</resultCode><resultMsg>\xec\xb5\x9c\xea\xb7\xbc 1\xec\x9d\xbc \xea\xb0\x84\xec\x9d\x98 \xec\x9e\x90\xeb\xa3\x8c\xeb\xa7\x8c \xec\xa0\x9c\xea\xb3\xb5\xed\x95\xa9\xeb\x8b\x88\xeb\x8b\xa4.</resultMsg></header></response>\r\n'


In [3]:
import requests
from datetime import datetime

# 공공 API의 서비스 키를 입력하세요
service_key = ''  # 여기에 실제 서비스 키를 입력하세요

# 현재 날짜와 시간 가져오기
now = datetime.now()
base_date = now.strftime('%Y%m%d')  # YYYYMMDD 형식
base_time = now.strftime('%H%M')    # HHMM 형식

# 서울시의 기상 정보 API 요청
url = ''
params = {
    'serviceKey': service_key,
    'pageNo': '1',
    'numOfRows': '1000',
    'dataType': 'XML',
    'base_date': base_date,
    'base_time': base_time,
    'nx': '55',  # 서울의 격자 좌표 nx
    'ny': '127'  # 서울의 격자 좌표 ny
}

response = requests.get(url, params=params)

# 요청 결과 출력
if response.status_code == 200:
    print(response.content)
else:
    print(f'Error: {response.status_code}')

b'<?xml version="1.0" encoding="UTF-8"?>\r\n<response><header><resultCode>00</resultCode><resultMsg>NORMAL_SERVICE</resultMsg></header><body><dataType>XML</dataType><items><item><baseDate>20240927</baseDate><baseTime>1300</baseTime><category>PTY</category><nx>55</nx><ny>127</ny><obsrValue>0</obsrValue></item><item><baseDate>20240927</baseDate><baseTime>1300</baseTime><category>REH</category><nx>55</nx><ny>127</ny><obsrValue>61</obsrValue></item><item><baseDate>20240927</baseDate><baseTime>1300</baseTime><category>RN1</category><nx>55</nx><ny>127</ny><obsrValue>0</obsrValue></item><item><baseDate>20240927</baseDate><baseTime>1300</baseTime><category>T1H</category><nx>55</nx><ny>127</ny><obsrValue>27</obsrValue></item><item><baseDate>20240927</baseDate><baseTime>1300</baseTime><category>UUU</category><nx>55</nx><ny>127</ny><obsrValue>0.2</obsrValue></item><item><baseDate>20240927</baseDate><baseTime>1300</baseTime><category>VEC</category><nx>55</nx><ny>127</ny><obsrValue>353</obsrValue><

In [4]:
import streamlit as st
import json
import requests
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableMap

# 기상 데이터 조회 함수
def get_weather_data(district):
    url = ''  # 기상 API 엔드포인트
    params = {
        'key': '',  # 기상 API 키
        'q': district,  # 사용자가 입력한 구
        'lang': 'ko'
    }
    response = requests.get(url, params=params)
    return json.loads(response.content)

# 메인 스트림릿 앱
def main():
    st.title("서울 구별 실시간 기상 정보 제공 챗봇")

    # 사용자 입력
    district = st.text_input("구 이름을 입력하세요:", "중랑구")
    
    if st.button("구 검색"):
        weather_data = get_weather_data(district)
        
        # 기상 정보 추출
        current_weather = weather_data['current']
        weather_info = {
            '위치': weather_data['location']['name'],
            '온도(°C)': current_weather['temp_c'],
            '습도(%)': current_weather['humidity'],
            '풍속(km/h)': current_weather['wind_kph'],
            '날씨': current_weather['condition']['text']
        }

        # 정보 표시
        st.write("현재 기상 정보:")
        for key, value in weather_info.items():
            st.write(f"{key}: {value}")

        # 챗봇 설정
        llm = ChatOllama(model="gemma2:9b", temperature=0.3)

        template = """
        당신은 기상정보를 안내하는 챗봇입니다. 
        사용자에게 가능한 많은 정보를 친절하게 제공하십시오.
        
        Answer the question as based only on the following context:
        {context}

        Question: {question}
        """
        prompt = ChatPromptTemplate.from_template(template)

        # 사용자 질문 처리
        question = st.text_input("궁금한 점을 입력하세요:")
        if question:
            context = f"{weather_info}"
            response = llm.invoke({"context": context, "question": question})
            st.markdown(response.content)

if __name__ == "__main__":
    main()


2024-09-27 13:45:06.286 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-27 13:45:06.759 
  command:

    streamlit run c:\Users\AISW-203-108\Desktop\gridone\venv0926\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-09-27 13:45:06.760 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-27 13:45:06.760 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-27 13:45:06.761 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-27 13:45:06.761 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-27 13:45:06.762 Session state does not function when running a script without `streamlit run`
2024-09-27 13:45:06.762 Thread 'MainThread': miss

In [8]:
import requests

# API 키를 여기에 입력하세요
api_key = 'F'  # 실제 API 키로 대체하세요
weather_data = get_weather_data(api_key)

def get_weather_data(api_key, location='서울'):
    url = ''  # WeatherAPI의 엔드포인트
    params = {
        'key': api_key,  # API 키
        'q': location,   # 요청할 지역
        'lang': 'ko'     # 언어 설정 (한국어)
    }

    response = requests.get(url, params=params)

    if response.status_code != 200:
        print(f"API 호출 오류: {response.status_code} - {response.text}")
        return None

    return response.json()  # JSON 형태로 응답 반환



if weather_data:
    # 기상 정보 출력
    location = weather_data['location']['name']
    temp_c = weather_data['current']['temp_c']
    condition = weather_data['current']['condition']['text']
    humidity = weather_data['current']['humidity']
    wind_kph = weather_data['current']['wind_kph']

    print(f"{location}의 현재 기온: {temp_c}°C")
    print(f"날씨 상태: {condition}")
    print(f"습도: {humidity}%")
    print(f"풍속: {wind_kph} km/h")

API 호출 오류: 500 - <?xml version="1.0" encoding="UTF-8"?>
<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/">
    <soapenv:Body>
        <soapenv:Fault>
            <faultcode>soapenv:Server</faultcode>
            <faultstring>Policy Falsified</faultstring>
            <faultactor>http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0?key=F2T8H%2Be5AtyrEGhN1h9WVc2vf92jzERErsq7S97z1iCzQna2FGFWRRj5BgkqNnYXifuOz%2FBSH2TAqHtpchBK6w%3D%3D&amp;q=%EC%84%9C%EC%9A%B8&amp;lang=ko</faultactor>
            <detail>
                <l7:policyResult
                    status="Service Not Found.  The request may have been sent to an invalid URL, or intended for an unsupported operation." xmlns:l7="http://www.layer7tech.com/ws/policy/fault"/>
            </detail>
        </soapenv:Fault>
    </soapenv:Body>
</soapenv:Envelope>



In [10]:
import requests
import xml.etree.ElementTree as ET

def get_weather_data():
    url = ''
    params = {
        'serviceKey': '',  # 여기에 실제 서비스 키를 입력하세요
        'pageNo': '1',
        'numOfRows': '1000',
        'dataType': 'XML',
        'base_date': '20210628',  # 이 날짜는 현재 날짜로 변경할 수 있습니다
        'base_time': '0600',  # 이 시간도 현재 시간으로 변경할 수 있습니다
        'nx': '55',  # 서울의 X좌표
        'ny': '127'  # 서울의 Y좌표
    }

    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        return response.content
    else:
        print("Error:", response.status_code)
        return None

def parse_weather_data(xml_data):
    root = ET.fromstring(xml_data)
    weather_info = []

    for item in root.findall('.//item'):
        category = item.find('category').text
        value = item.find('obsrValue').text
        weather_info.append((category, value))

    return weather_info

def main():
    xml_data = get_weather_data()
    
    if xml_data:
        weather_info = parse_weather_data(xml_data)
        
        # 서울의 기상정보 출력
        for category, value in weather_info:
            print(f"{category}: {value}")

if __name__ == "__main__":
    main()
